In [ ]:
%matplotlib inline

from __future__ import print_function

import models.graph_models as md
import data.data_cost as dt
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch

In [ ]:
data = dt.load_dataset('../saved/data_040519.data.sample1000')

In [ ]:
params = md.RnnParameters(256, 256, 1, False, False, False, md.RnnHierarchyType.MULTISCALE, md.RnnType.LSTM)
model = md.RNN(params)
model.set_learnable_embedding('none', 628)
state_dict = torch.load('../saved/040519_lstm_decay_trained.mdl')
model.load_state_dict(state_dict['model'])

In [ ]:
def get_embedding(instr_x):
    token_state = model.get_token_init()
    tokens = model.final_embeddings(torch.LongTensor(instr_x)).unsqueeze(1)
    _, (output, _) = model.token_rnn(tokens, token_state)
    return output.squeeze()

def predict(X):
    token_embeddings = list(map(get_embedding, X))
    instr_chain = torch.stack(token_embeddings).unsqueeze(1)
    # TODO: permute embeddings?
    _, (final_state, _) = model.instr_rnn(instr_chain, model.get_instr_init())
    return model.linear(final_state.squeeze()).squeeze()

In [ ]:
def adj_mtx_of_block(block):
    idxmap = {instr: i for (i, instr) in enumerate(block.instrs)}
    m = np.zeros((len(idxmap), len(idxmap)), dtype=np.float32)
    for (i, instr) in enumerate(block.instrs):
        for ch in instr.children:
            m[i, idxmap[ch]] = 1
    return m

In [ ]:
print('Block\n{}'.format('='*40))
print('\n'.join(instr.intel for instr in data.data[0].block.instrs))
print('\nDependency matrix\n{}'.format('='*40))
print(adj_mtx_of_block(data.data[0].block))
print('\nPredicted runtime\n{}'.format('='*40))
print('Predicted time: {}'.format(predict(data.data[0].x).item()))
print('\nDAG\n{}'.format('='*40))
data.data[0].block.draw()